In [1]:
#import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [2]:
#get the model
model = TFBertForSequenceClassification.from_pretrained('./BERT/MonoBERTEmotion', num_labels = 5)
tokenizer = BertTokenizer.from_pretrained('./bert_model')
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./BERT/MonoBERTEmotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  135193344 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 135,197,189
Trainable params: 135,197,189
Non-trainable params: 0
_________________________________________________________________


In [3]:
#read the data
test = pd.read_csv('EmotionTest.csv', encoding = 'utf-8')

In [4]:
test.head()

,Unnamed: 0,Unnamed: 0.1,ID,Tweet,Emotions
0,0,0,2018-En-00866,"@RanaAyyub @rajnathsingh Օ՜, թաքնված վրեժ և զա...",anger
1,1,1,2018-En-02590,"Ես այս ամենն անում եմ, որպեսզի համոզվեմ, որ դո...",joy
2,2,2,2018-En-03361,"եթե ոչ, ապա #teamchristine bc այն ամենը, ինչ Տ...",anger
3,3,3,2018-En-03230,Դա #հիանալի սկիզբ է #սկսնակների համար՝ անցնելո...,joy
4,4,4,2018-En-01143,Իմ լավագույն ընկերներն առաջին անգամ ինձ հետ վա...,fear


In [5]:
#create a polarity column and change the targets accordingly
test['Polarity'] = np.nan
for i in range(len(test)):
    if test['Emotions'][i] == 'anger':
        test['Polarity'][i] = 0
    elif test['Emotions'][i] == 'fear':
        test['Polarity'][i] = 1
    elif test['Emotions'][i] == 'joy':
        test['Polarity'][i] = 2
    elif test['Emotions'][i] == 'sadness':
        test['Polarity'][i] = 3
    else:
        test['Polarity'][i] = 4
test = test.drop('Emotions', axis = 1)

<ipython-input-5-a0aa1379ddaf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Polarity'][i] = 0
<ipython-input-5-a0aa1379ddaf>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Polarity'][i] = 2
<ipython-input-5-a0aa1379ddaf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Polarity'][i] = 1
<ipython-input-5-a0aa1379ddaf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



In [6]:
#predict
pred_sentences = test['Tweet']
tf_batch = tokenizer(list(pred_sentences), max_length = 128, padding = True, truncation = True, return_tensors = 'tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis = -1)
labels = [0, 1, 2, 3, 4]
label = tf.argmax(tf_predictions, axis = 1)
label = label.numpy()
predictions = pd.Series(label, index = test.index)
print('Accuracy:', accuracy_score(test['Polarity'], predictions))
print('F-measure:', f1_score(test['Polarity'], predictions, average = None))
print('Recall:', recall_score(test['Polarity'], predictions, average = None))
print('Precision:', precision_score(test['Polarity'], predictions, average = None))

Accuracy: 0.7743682310469314
F-measure: [0.70542636 0.60215054 0.87223169 0.66257669 0.        ]
Recall: [0.68421053 0.65116279 0.8707483  0.69230769 0.        ]
Precision: [0.728      0.56       0.87372014 0.63529412 0.        ]


In [7]:
from sklearn.metrics.cluster import contingency_matrix
contingency_matrix(test['Polarity'], predictions)

array([[ 91,  12,  21,   8,   1],
       [  6,  28,   7,   2,   0],
       [ 13,   6, 256,  19,   0],
       [ 14,   3,   7,  54,   0],
       [  1,   1,   2,   2,   0]])

In [8]:
import pickle
with open('EmotionTestText.pickle', 'rb') as f:
    lemmatized_text = pickle.load(f)

In [9]:
tf_predictions

<tf.Tensor: shape=(554, 5), dtype=float32, numpy=
array([[0.87981033, 0.05415289, 0.0405109 , 0.00547917, 0.02004664],
       [0.00204022, 0.00197981, 0.9849223 , 0.00740052, 0.0036571 ],
       [0.78637516, 0.13120304, 0.05558843, 0.00500116, 0.02183221],
       ...,
       [0.37149763, 0.04411145, 0.35387027, 0.09931082, 0.13120984],
       [0.00733033, 0.0282471 , 0.7233638 , 0.208778  , 0.03228078],
       [0.00210433, 0.00221007, 0.9873433 , 0.00394196, 0.00440025]],
      dtype=float32)>

In [10]:
h = np.zeros((len(tf_predictions), 2))

for i in range(len(tf_predictions)):
    h[i][0] = tf_predictions[i][0] + tf_predictions[i][1] + tf_predictions[i][3]
    h[i][1] = tf_predictions[i][2] + tf_predictions[i][4]

In [11]:
x = []

for i in range(len(h)):
    r = h[i]
    if r[0] > r[1]:
        x.append(r[0] / r[1])
    else:
        x.append(r[1] / r[0])

In [12]:
x

[15.513219135260782,
 86.56148172044729,
 11.916452182770017,
 106.77922621001987,
 65.86950411003812,
 41.77938758331238,
 43.69825962049291,
 55.944537748455076,
 71.30136584130815,
 75.62923798592495,
 1.1317121162202055,
 11.622529421794345,
 147.14290342154416,
 112.23827517526095,
 30.421769835035256,
 124.73368658084898,
 6.964015707570512,
 159.36856350572293,
 159.00568752908623,
 11.650690601655098,
 1.0082285795955142,
 58.82492355166027,
 35.9499213025246,
 42.78252257747781,
 25.42558989990711,
 107.04350571792253,
 26.572779852424997,
 11.278404398857107,
 23.1388246880577,
 47.812735166635996,
 87.50961730747703,
 9.56503288187823,
 4.303438639970159,
 169.04150089309087,
 2.6173789984947007,
 26.97643005143169,
 170.61355788466938,
 64.05938526037401,
 83.62959015084556,
 40.35297961693775,
 25.954676339612956,
 6.193039515971367,
 168.44826655666188,
 93.96288375907096,
 1.165824183101626,
 157.66940917486397,
 9.179573310742013,
 14.269423309552812,
 45.36396175097927

In [13]:
possibility = pd.Series(x, index = test.index)

In [14]:
index = []
for i in range(len(predictions)):
    if predictions[i] != test['Polarity'][i]:
        index.append(i)

y = {'Text': test['Tweet'].loc[index], 'Label': test['Polarity'].loc[index], 'Prediction': predictions[index], 'Possibility': possibility[index]}
summary = pd.DataFrame(y)
summary

,Text,Label,Prediction,Possibility
4,Իմ լավագույն ընկերներն առաջին անգամ ինձ հետ վա...,1.0,0,65.869504
10,"Օ, մելամաղձոտ կատակոմբները արագ թափառեցին Մորգ...",3.0,2,1.131712
11,"#LouiseLinton - ատողները ատում են, շարունակելո...",3.0,0,11.622529
14,"#RIPBiwott Կարծում եմ, որ Ռոբերտ Ուկոսի հոգին ...",0.0,2,30.421770
16,@GarfieldLineker @TimCRoberts *իր մեջքին: Ներո...,3.0,0,6.964016
...,...,...,...,...
536,@LogicIsPowerful Ոչ բոլորն են վատը: Բայց ես պա...,3.0,0,9.322150
537,"Այն ամենը, ինչ ես պատվիրում եմ առցանց, պարզապե...",0.0,2,1.014781
541,@uzalu_ @Veeh_Ro Ի՜նչ անուրախ քիթ է։,0.0,3,4.318763
543,Այսօրվա համար շատ կարևոր բան. \n\nՄի՛ ահաբեկեք...,3.0,1,4.716560


In [15]:
def get_index(cont, thr):
    ind = []
    for i in range(len(cont)):
        if cont[i] < thr:
            ind.append(i)
    return ind

In [16]:
low_index = get_index(possibility, 1.3)

In [17]:
low_index

[10,
 20,
 44,
 76,
 81,
 106,
 108,
 144,
 153,
 164,
 181,
 193,
 252,
 292,
 318,
 353,
 392,
 429,
 435,
 441,
 452,
 460,
 479,
 483,
 485,
 506,
 525,
 537,
 546,
 551]

In [18]:
len(test)

554

In [19]:
len(low_index)

30

In [20]:
with open('anger_contribution.pickle', 'rb') as f:
    anger_contribution = pickle.load(f)
    
with open('fear_contribution.pickle', 'rb') as f:
    fear_contribution = pickle.load(f)

with open('joy_contribution.pickle', 'rb') as f:
    joy_contribution = pickle.load(f)
    
with open('sadness_contribution.pickle', 'rb') as f:
    sadness_contribution = pickle.load(f)

with open('surprise_contribution.pickle', 'rb') as f:
    surprise_contribution = pickle.load(f)
    
with open('positive_adverb.pickle', 'rb') as f:
    positive_adverb = pickle.load(f)
    
with open('negative_adverb.pickle', 'rb') as f:
    negative_adverb = pickle.load(f)

In [21]:
anger_contribution

,Word,Count,Contribution
0,ազգայնամոլ,0,0.500000
1,ազգայնամոլություն,0,0.500000
2,ազգանվանանակից,0,0.500000
3,ազգություն,0,0.500000
4,ակոս,0,0.500000
...,...,...,...
612,օրենսդրական,0,0.500000
613,օրենականություն,0,0.500000
614,օրենսգիրք,0,0.500000
615,օրինագիծ,1,0.731059


In [22]:
fear_contribution

,Word,Count,Contribution
0,միանգամայն բավարար,0,0.500000
1,լիառատ,0,0.500000
2,նոր,426,1.000000
3,թթվային,0,0.500000
4,ագահ,11,0.999983
...,...,...,...
6535,օրորել,1,0.731059
6536,օրհասական,0,0.500000
6537,ֆանտազիա,1,0.731059
6538,ֆեդերալ,1,0.731059


In [23]:
joy_contribution

,Word,Count,Contribution
0,ագահություն,0,0.500000
1,ագրեսիվ,1,0.731059
2,ազդու,0,0.500000
3,ակամայից,2,0.880797
4,աղբի,0,0.500000
...,...,...,...
528,օճառոտ,0,0.500000
529,օճառ քսել,0,0.500000
530,օրացույց,0,0.500000
531,օրինակել,2,0.880797


In [24]:
sadness_contribution

,Word,Count,Contribution
0,ազգաբնակչություն,0,0.500000
1,ախտորոշել,0,0.500000
2,ածել,105,1.000000
3,ականանետ,0,0.500000
4,ակամա,6,0.997527
...,...,...,...
474,քրոնիկ,1,0.731059
475,օդափոխել,0,0.500000
476,օթյակ,0,0.500000
477,օջախ(տուն),0,0.500000


In [25]:
surprise_contribution

,Word,Count,Contribution
0,ազատարար,0,0.500000
1,ազդել,5,0.993307
2,ազդեցիկ,0,0.500000
3,ազդեցություն,3,0.952574
4,ազնվաքիս,0,0.500000
...,...,...,...
997,օղակ,3,0.952574
998,օսմանյան,0,0.500000
999,օրհնել,3,0.952574
1000,օրորոց,0,0.500000


In [26]:
positive_adverb

['ամբողջությամբ',
 'անպայման',
 'խնդրեմ',
 'նույնիսկ',
 'դեռ',
 'դեռևս',
 'արդեն',
 'անկասկած',
 'շատ',
 'բարձրաձայն',
 'հատկապես',
 'պասսիվորեն',
 'հեշտորեն',
 'հեշտությամբ',
 'քննադատաբար',
 'քաղաքավարիորեն',
 'համեստորեն',
 'զգույշ',
 'քաոսայնորեն',
 'հերոսաբար',
 'սառնորեն',
 'ամբողջովին',
 'ամբողջովին ',
 'անխտիր',
 'ավելի լավ',
 'ավելի քիչ',
 'բոլորովին',
 'զարմանալիորեն',
 'էլ',
 'ինչքան էլ որ',
 'լիովին',
 'կատարելապես',
 'հաճախ',
 'համապատասխանաբար',
 'հենց',
 'հետեւաբար',
 'հոգեպես',
 'միանգամայն',
 'շուրջ',
 'որքան էլ որ',
 'պարզապես',
 'վճռականորեն',
 'տրամաբանորեն',
 'ուղակի']

In [27]:
negative_adverb

['ուղղակի',
 'երբեք',
 'չ-',
 'շուտով',
 'շուտ',
 'հազվադեպ',
 'իհարկե',
 'անշուշտ',
 'սովորաբար',
 'չափազանց',
 'սաստիկ',
 'քիչ',
 'սակավ',
 'ֆիզիկապես',
 'ֆիզիկորեն',
 'այժմ',
 'անմիջապես',
 'հիմա',
 'պատահաբար']

In [28]:
test.loc[low_index]

,Unnamed: 0,Unnamed: 0.1,ID,Tweet,Polarity
10,10,10,2018-En-01472,"Օ, մելամաղձոտ կատակոմբները արագ թափառեցին Մորգ...",3.0
20,20,23,2018-En-01237,#Թրամփի միակ #մտահոգությունը անձնական #շահույթ...,0.0
44,44,63,2018-En-00685,"Ուրախ եմ, որ մեկ անգամ թիմով խաղացինք tgt: Մտա...",3.0
76,76,107,2018-En-03321,"#Աշխարհ գիտե՞ս, թե որն է տարբերությունը #հարբա...",2.0
81,81,119,2018-En-01525,Շնորհակալություն հայտնելու կարիք չկա: 😃,2.0
106,106,159,2018-En-00621,"արագ նշում insta story-ների մասին, թե ինչպես է...",0.0
108,108,165,2018-En-02533,"Մի նշիր ինձ նկարներում որպես «նախ ընտանիքը», ե...",0.0
144,144,219,2018-En-01458,"Հիշեք, որ ձեր ինքնությունը Քրիստոսի մեջ է: Մեր...",1.0
153,153,237,2018-En-02101,"Tipoftheday. Մի շեղվեք նրանց կողմից, ովքեր մեղ...",2.0
164,164,259,2018-En-00755,Ինչպե՞ս եք վերաբերվում @Snapchat-ի նոր ֆունկցի...,2.0


In [29]:
lemmatized_text = lemmatized_text.reset_index()

In [30]:
lemmatized_text

,index,0
0,0,"[օ, թաքնվել, վրեժ, և, զայրույթ, հիշել, ժամանակ..."
1,1,"[ամենը, անել, համոզվել, դու, ժպտալ, եղբայր]"
2,2,"[ոչ, տանել, անել, սադրել, թվիթեր, ստվերային, բ..."
3,3,"[հիանալի, սկիզբ, սկսնակ, անցնել, ավտ, առևտուր,..."
4,4,"[լավ, ընկեր, առաջին, անգամ, հետ, վարել, մեքենա..."
...,...,...
549,877,"[երբեք, զղջա, բան, որ, ժամանակին, ստիպել, ժպտա..."
550,880,"[մեկը, կողմ, խորապես, սիրել, լինել, ուժ, տալ, ..."
551,881,"[չ, ծիծաղել, թվիթ]"
552,882,"[ուրախ, դիտել, երեկո, գալ]"


In [31]:
ang_count = []
fea_count = []
joy_count = []
sad_count = []
sur_count = []
data = lemmatized_text.iloc[low_index]

for i in low_index:
    words = data[0][i]
    ang = 0
    fea = 0
    joy = 0
    sad = 0
    sur = 0
    for j in range(len(words)):
        if words[j] in anger_contribution['Word'].values:
            ang += 1
        elif words[j] in fear_contribution['Word'].values:
            fea += 1
        elif words[j] in joy_contribution['Word'].values:
            joy += 1
        elif words[j] in sadness_contribution['Word'].values:
            sad += 1
        elif words[j] in surprise_contribution['Word'].values:
            sur += 1
    ang_count.append(ang)
    fea_count.append(fea)
    joy_count.append(joy)
    sad_count.append(sad)
    sur_count.append(sur)

In [32]:
ang_count

[0,
 0,
 0,
 0,
 1,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0]

In [33]:
fea_count

[2,
 8,
 12,
 3,
 1,
 3,
 6,
 6,
 8,
 4,
 0,
 5,
 1,
 9,
 2,
 3,
 1,
 3,
 2,
 6,
 5,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 1]

In [34]:
joy_count

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [35]:
sad_count

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [36]:
sur_count

[0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0]

In [37]:
def found_something(ind, words, window, adverb):
    a = ind - 1
    found_a = False
    count_a = 0
    while a >= 0 and found_a == False and count_a <= window:
        if words[a] in adverb:
            found_a = True
        a = a - 1
        count_a = count_a + 1
    b = ind + 1
    found_b = False
    count_b = 0
    while b < len(data[0][i]) and found_b == False and count_b <= window:
        if words[b] in adverb:
            found_b = True
        b = b + 1
        count_b = count_b + 1
    return found_a, found_b

In [38]:
anger_emotion = []
fear_emotion = []
joy_emotion = []
sadness_emotion = []
surprise_emotion = []

for i in low_index:
    words = data[0][i]
    ang_words = []
    fea_words = []
    joy_words = []
    sad_words = []
    sur_words = []
    ang_scores = np.array([])
    fea_scores = np.array([])
    joy_scores = np.array([])
    sad_scores = np.array([])
    sur_scores = np.array([])
    
    for j in range(len(words)):
        if words[j] in anger_contribution['Word'].values:
            ang_ind = np.where(anger_contribution['Word'] == words[j])[0]
            ang_cont = anger_contribution['Contribution'][ang_ind]
            ang_words.append([words[j], ang_cont])
        elif words[j] in fear_contribution['Word'].values:
            fea_ind = np.where(fear_contribution['Word'] == words[j])[0]
            fea_cont = fear_contribution['Contribution'][fea_ind]
            fea_words.append([words[j], fea_cont])
        elif words[j] in joy_contribution['Word'].values:
            joy_ind = np.where(joy_contribution['Word'] == words[j])[0]
            joy_cont = joy_contribution['Contribution'][joy_ind]
            joy_words.append([words[j], joy_cont])
        elif words[j] in sadness_contribution['Word'].values:
            sad_ind = np.where(sadness_contribution['Word'] == words[j])[0]
            sad_cont = sadness_contribution['Contribution'][sad_ind]
            sad_words.append([words[j], sad_cont])
        elif words[j] in surprise_contribution['Word'].values:
            sur_ind = np.where(surprise_contribution['Word'] == words[j])[0]
            sur_cont = surprise_contribution['Contribution'][sur_ind]
            sur_words.append([words[j], sur_cont])
            
    for k in range(len(ang_words)):
        word = ang_words[k][0]
        c = ang_words[k][1]
        ind = data[0][i].index(word)
        pos_found = found_something(ind, words, 4, positive_adverb)
        if pos_found[0] == True or pos_found[1] == True:
            g = 2
        else:
            g = 1
        neg_found = found_something(ind, words, 3, negative_adverb)
        if neg_found[0] == True or neg_found[1] == True:
            f = 1
        else:
            f = -1
        ang_scores = np.append(ang_scores, g * f * c)
        
    for l in range(len(fea_words)):
        word = fea_words[l][0]
        c = fea_words[l][1]
        ind = data[0][i].index(word)
        pos_found = found_something(ind, words, 4, positive_adverb)
        if pos_found[0] == True or pos_found[1] == True:
            g = 2
        else:
            g = 1
        neg_found = found_something(ind, words, 3, negative_adverb)
        if neg_found[0] == True or neg_found[1] == True:
            f = 1
        else:
            f = -1
        fea_scores = np.append(fea_scores, g * f * c)

    for m in range(len(joy_words)):
        word = joy_words[m][0]
        c = joy_words[m][1]
        ind = data[0][i].index(word)
        pos_found = found_something(ind, words, 4, positive_adverb)
        if pos_found[0] == True or pos_found[1] == True:
            g = 2
        else:
            g = 1
        neg_found = found_something(ind, words, 3, negative_adverb)
        if neg_found[0] == True or neg_found[1] == True:
            f = 1
        else:
            f = -1
        joy_scores = np.append(joy_scores, g * f * c)
        
    for n in range(len(sad_words)):
        word = sad_words[n][0]
        c = sad_words[n][1]
        ind = data[0][i].index(word)
        pos_found = found_something(ind, words, 4, positive_adverb)
        if pos_found[0] == True or pos_found[1] == True:
            g = 2
        else:
            g = 1
        neg_found = found_something(ind, words, 3, negative_adverb)
        if neg_found[0] == True or neg_found[1] == True:
            f = 1
        else:
            f = -1
        sad_scores = np.append(sad_scores, g * f * c)
    
    for o in range(len(sur_words)):
        word = sur_words[o][0]
        c = sur_words[o][1]
        ind = data[0][i].index(word)
        pos_found = found_something(ind, words, 4, positive_adverb)
        if pos_found[0] == True or pos_found[1] == True:
            g = 2
        else:
            g = 1
        neg_found = found_something(ind, words, 3, negative_adverb)
        if neg_found[0] == True or neg_found[1] == True:
            f = 1
        else:
            f = -1
        sur_scores = np.append(sur_scores, g * f * c)
    
    anger_emotion.append(np.sum(ang_scores))
    fear_emotion.append(np.sum(fea_scores))
    joy_emotion.append(np.sum(joy_scores))
    sadness_emotion.append(np.sum(sad_scores))
    surprise_emotion.append(np.sum(sur_scores))

In [39]:
for i in range(len(low_index)):
    scores = np.array([anger_emotion[i], fear_emotion[i], joy_emotion[i], sadness_emotion[i], surprise_emotion[i]])
    maximum_index = np.argmax(scores)
    predictions[low_index[i]] = maximum_index

In [40]:
print('Accuracy:', accuracy_score(test['Polarity'], predictions))
print('F-measure:', f1_score(test['Polarity'], predictions, average = None))
print('Recall:', recall_score(test['Polarity'], predictions, average = None))
print('Precision:', precision_score(test['Polarity'], predictions, average = None))

Accuracy: 0.7707581227436823
F-measure: [0.69784173 0.60416667 0.8712522  0.67080745 0.        ]
Recall: [0.72932331 0.6744186  0.84013605 0.69230769 0.        ]
Precision: [0.66896552 0.54716981 0.9047619  0.65060241 0.        ]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
contingency_matrix(test['Polarity'], predictions)

array([[ 97,  13,  15,   8],
       [  8,  29,   4,   2],
       [ 23,   7, 247,  17],
       [ 16,   3,   5,  54],
       [  1,   1,   2,   2]])